In [0]:
%pip install databricks-feature-store

In [0]:
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
import databricks.feature_store as feature_store
from databricks.feature_store import FeatureStoreClient
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import joblib


In [0]:
data=pd.read_csv('/Workspace/Users/pratik.potdar@infocepts.com/mlops/mlops_input_data.csv')

In [0]:
data.head()

In [0]:
data.drop(['Unnamed: 0'],axis=1,inplace=True)

In [0]:
data.describe()

In [0]:
data.info()

In [0]:
num_cols=data.select_dtypes(include=['float64','int'])


In [0]:
num_cols.head()

In [0]:
num_cols.columns

In [0]:
#check nan values

In [0]:
data[num_cols.columns].isnull().sum()

In [0]:
#fill the meaningful information into nan values

In [0]:
data['total_value_remittance_deposits'].fillna(0, inplace=True)

In [0]:
data['total_num_remittance_deposits'].fillna(0, inplace=True)

In [0]:
mean_values = data.groupby('had_debcard').mean()

In [0]:
mean_values['day_since_last_withdrawal']

In [0]:
data['day_since_last_withdrawal'] = data['day_since_last_withdrawal'].fillna(data['had_debcard'].map({0: 72, 1: 21}))

In [0]:
corr=data.corr().abs()

In [0]:
corr.unstack().sort_values(ascending=False).head(50)

In [0]:
high_corr=list(corr['had_debcard'].sort_values(ascending=False).head(10).index)

In [0]:
##Categorical column

In [0]:
cat_cols=data.select_dtypes(include=['object'])


In [0]:
cat_cols.drop(['business_id'],axis=1,inplace=True)

In [0]:
cat_cols.nunique()

In [0]:
cat_cols.isna().sum()

In [0]:
cat_cols.columns

In [0]:
cat_cols['operating_address_city'].fillna('unknown', inplace=True)

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


In [0]:
label_cols=['gender','nationality'] 

In [0]:
data[label_cols].head()

In [0]:
cat_cols['gender'].nunique()

In [0]:
label_encoders = {}

In [0]:
for column in label_cols:
   le = LabelEncoder()
   cat_cols[column] = le.fit_transform(cat_cols[column])
   label_encoders[column] = le
# Save the label encoders to use when deploying the model
joblib.dump(label_encoders, '/Workspace/Users/pratik.potdar@infocepts.com/mlops/label_encode.pkl')

In [0]:
#label_encoders = {}
#for col in label_cols:
    #le = LabelEncoder()
    #cat_cols[col] = le.fit_transform(cat_cols[col])
    #label_encoders[col] = le
    # Save the label encoder for future use
    #joblib.dump(le, f'/Workspace/Users/pratik.potdar@infocepts.com/mlops/{col}#_label_encoder.pkl')

In [0]:
#cat_cols = pd.get_dummies(cat_cols, columns=['operating_address_city', 'subscription_plan', 'industry_subsector_category'])

In [0]:
one_hot_cols=['operating_address_city', 'subscription_plan', 'industry_subsector_category']

In [0]:
one_hot_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

In [0]:
cat_cols.head()

In [0]:
one_hot_encoded = one_hot_encoder.fit_transform(cat_cols[one_hot_cols])

In [0]:
one_hot_encoded

In [0]:
cat_cols = cat_cols.drop(one_hot_cols, axis=1)

In [0]:
cat_cols = pd.concat([cat_cols, pd.DataFrame(one_hot_encoded)], axis=1)

In [0]:
joblib.dump(one_hot_encoder, '/Workspace/Users/pratik.potdar@infocepts.com/mlops/one_hot_encoder.pkl')

In [0]:
cat_cols.head()

In [0]:
out_columns=['total_value_remittance','total_value_cash_deposit','total_num_remittance_deposits','day_since_last_withdrawal','latest_eod_balance','ca_avg_daily_balance_l30d']

In [0]:
for col in out_columns:
    data[col] = np.log1p(data[col]) 

In [0]:
total=pd.concat([data[list(num_cols.columns)], cat_cols], axis=1)

In [0]:
total.drop(columns=['had_debcard'],axis=1,inplace=True)

In [0]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(total)
total = pd.DataFrame(scaled_data, columns=total.columns)

In [0]:

joblib.dump(scaler, '/Workspace/Users/pratik.potdar@infocepts.com/mlops/scaler.pkl')

In [0]:
data.columns

In [0]:
total['business_id']=data['business_id']
total['had_debcard']=data['had_debcard']

In [0]:
fs = FeatureStoreClient()

In [0]:
new=spark.createDataFrame(total) 

In [0]:
fs.create_table(
    name="Preprocessing",   # Name of the feature table
    primary_keys=["business_id"],                       # Primary keys for the table
    df=spark.createDataFrame(total),                                     # The DataFrame to store
    description="This table contains the preprocessed data"
)